In [1]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
input_fname = 'test_IOTA.in'
np.set_printoptions(precision=3)

# Prepare Beam and Lattice

In [2]:
beam, lattice0 = impact.readInputFile(input_fname)

reading ImpactZ input file (test_IOTA.in)
  : mpi task info .............................done
  : simulation control parameters .............done
  : space charge field solver, mesh info ......done
  : dist-type,restart,subcycle,#of state ......done
  : Multiple Charge State info ................done
  : particle distribution info ................done
  : beam reference orbit info .................done
  : converting impact dist to twiss param......done
  : lattice info ..............................done


In [3]:
beam.nCore_y=1
beam.nCore_z=1
beam.n_particles=2**15
beam.current=0.1
dist = beam.distribution
dist.distribution_type = 'Waterbag'
lattice = [impact.getElem('loop_through_lattice'),
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl'),
           0,
           impact.getElem('write_raw_ptcl')]
lattice[0].nturn = 1
lattice[1].file_id = 1001
lattice[3].file_id = 1002
lattice[5].file_id = 1003


ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

# 1. Drift

In [4]:
elemF = impact.getElem('drift')
elemF.length = 0.02
elemF.n_sckick = 20
elemF.n_map = 1

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

input error <- sum(beam.multi_charge.n_particles) not qual to beam.n_particles
  ... enforcing  beam.multi_charge.n_particles[0] to beam.n_particles
input error <- beam.multi_charge.current[0] not qual to beam.current
  ... enforcing  beam.multi_charge.current[0] to beam.current


0

In [5]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:8,:6]-pData2[:8,:6])

[[ 0.000e+00  0.000e+00  0.000e+00  2.168e-19  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.735e-18 -1.137e-13  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  5.684e-14  0.000e+00]
 [ 0.000e+00  0.000e+00  8.674e-19  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 5.421e-19  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  2.168e-19 -2.168e-19  0.000e+00  0.000e+00  0.000e+00]]


# 2. Quad

In [6]:
elemF = impact.getElem('quad')
elemF.length = 0.02
elemF.n_sckick = 20
elemF.n_map = 1
elemF.B1 = 15.0

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [7]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:8,:6]-pData2[:8,:6])

[[ 4.337e-18  1.735e-18 -4.879e-19  1.843e-18  0.000e+00  0.000e+00]
 [-1.735e-18 -8.674e-19  6.505e-18 -6.505e-18  0.000e+00  0.000e+00]
 [-3.036e-18 -1.626e-19  3.036e-18 -3.903e-18  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -4.337e-18  5.204e-18  0.000e+00  0.000e+00]
 [-2.168e-19  4.337e-19 -9.758e-19  4.554e-18  0.000e+00  0.000e+00]
 [ 0.000e+00 -4.337e-19  1.694e-20 -2.168e-18  0.000e+00  0.000e+00]
 [-1.084e-19  1.301e-18  3.469e-18 -4.120e-18  0.000e+00  0.000e+00]
 [ 1.626e-19  1.084e-18  2.385e-18 -8.132e-19  0.000e+00  0.000e+00]]


# 3. Sextupole

In [8]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'multipole_thin':
        break
lattice0[i]


 KL_dipole: 0.0
   KL_quad: 0.0
   KL_sext: 0.15
    KL_oct: 0.0
   KL_deca: 0.0
 KL_dodeca: 0.0
      type: 'multipole_thin'

In [9]:
elemF = lattice0[i]
elemF.KL_sext = 1000.0

elemB = copy(elemF)
elemB.KL_sext =-elemF.KL_sext

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [10]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:8,:6]-pData2[:8,:6])

[[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 2.168e-19 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
 [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]]


# 4. Dipole

In [11]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'dipole':
        break
lattice0[i]


                   length: 0.7330382858 [m]
                 n_sckick: 10 [1]
                    n_map: 20 [1]
            bending_angle: 1.047197551 [rad]
                       k1: 0.0
                  file_id: 350
              pipe_radius: 0.058 [m]
           entrance_angle: 0.0 [rad]
               exit_angle: 0.0 [rad]
       entrance_curvature: 0.0 [rad]
           exit_curvature: 0.0 [rad]
 fringe_field_integration: 0.5 [1]
                     type: 'dipole'

### negative length, bending angle and FINT

In [12]:
elemF = copy(lattice0[i])

elemB = copy(elemF)
elemB.length = -elemF.length
elemB.bending_angle = -elemF.bending_angle
elemB.fringe_field_integration = -elemF.fringe_field_integration

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

0

In [13]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:8,:6]-pData2[:8,:6])

[[-9.541e-18  3.469e-18  0.000e+00 -2.168e-19  7.105e-15  0.000e+00]
 [ 4.770e-18 -2.602e-18  0.000e+00  1.301e-18  0.000e+00  0.000e+00]
 [-1.735e-18 -2.331e-18  1.735e-18 -1.301e-18  0.000e+00  0.000e+00]
 [ 0.000e+00  2.819e-18 -4.337e-19  0.000e+00  0.000e+00  0.000e+00]
 [ 6.505e-19  1.735e-18 -1.626e-19  4.337e-19  0.000e+00  0.000e+00]
 [ 1.735e-18 -2.168e-19  2.406e-19 -8.674e-19  0.000e+00  0.000e+00]
 [-2.494e-18 -2.602e-18  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.626e-19 -1.952e-18 -6.505e-19  5.421e-19  0.000e+00  0.000e+00]]


# 5. Nonlinear Insert

In [14]:
for i in range(len(lattice0)):
    if lattice0[i].type == 'nonlinear_insert':
        break
lattice0[i]


             length: 1.8 [m]
           n_sckick: 96 [1]
              n_map: 5 [1]
         strength_t: 0.4
 transverse_scale_c: 0.01
       tune_advance: 0.3034496449165134
        pipe_radius: 10.0 [m]
               type: 'nonlinear_insert'

In [ ]:
elemF = lattice0[i]

elemB = copy(elemF)
elemB.length =-elemF.length

lattice[2] = elemF
lattice[4] = elemB

impact.writeInputFile(beam,lattice);
impact.run()

In [ ]:
pData0 = impact.readParticleData(1001,ke,mass,freq)
pData1 = impact.readParticleData(1002,ke,mass,freq)
pData2 = impact.readParticleData(1003,ke,mass,freq)
print(pData0[:8,:6]-pData2[:8,:6])